# Multi Class Classifier Using Feature Selection - Wrapper

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

from sklearn.svm import SVC
from sklearn.utils import shuffle

train_set = pd.read_csv('../Small Training Set.csv', header=None)

# train_set = pd.read_csv('../20 Percent Training Set.csv', header=None)
# train_set = pd.read_csv('../KDDTrain+.csv', header=None)

fn = pd.read_csv('../Field Names.csv', header=None)

fn2 = pd.DataFrame([['attack_type', 'symbolic'],['??','continuous']])
fn = fn.append(fn2, ignore_index=True)

cont_feats = fn.loc[fn[1] == 'continuous']
cont_feats = cont_feats.drop(columns=[1])

fn = fn.drop(columns=[1])

dfhd = fn[0].tolist()
train_set.columns = dfhd

cols = train_set.columns.tolist()
cols = cols[-2:-1] + cols[:-2] + cols[-1:]
train_set = train_set[cols]

att_types = train_set['attack_type']

# train_set.sort_values('attack_type')

# *Enumeration of Categoricals*

In [2]:
# ENUMERATING SERVICE TYPE
import copy

service_list = train_set['service'].unique()

try:
    service_list2
except NameError:
    service_list2 = copy.deepcopy(service_list)    
    svc_enum_data = {'service': service_list}
    service_enumerations = pd.DataFrame(data=svc_enum_data)
    service_enumerations.to_csv('Service_enums.csv')

enumerated_service_list = [x for x in range(len(service_list))]


train_set = train_set.replace(service_list,enumerated_service_list)


In [3]:
# ENUMERATING PROTOCOL TYPE

prot_list = train_set['protocol_type'].unique()

try:
    prot_list2
except NameError:
    prot_list2 = copy.deepcopy(prot_list)    
    prot_enum_data = {'protocol_type': prot_list}
    prot_enumerations = pd.DataFrame(data=prot_enum_data)
    prot_enumerations.to_csv('Prot_enums.csv')

enumerated_prot_list = [x for x in range(len(prot_list))]


train_set = train_set.replace(prot_list,enumerated_prot_list)


In [4]:
# ENUMERATING FLAG TYPE

flag_list = train_set['flag'].unique()

try:
    flag_list2
except NameError:
    flag_list2 = copy.deepcopy(flag_list)    
    flag_enum_data = {'flag': flag_list}
    flag_enumerations = pd.DataFrame(data=flag_enum_data)
    flag_enumerations.to_csv('Flag_enums.csv')

enumerated_flag_list = [x for x in range(len(flag_list))]


train_set = train_set.replace(flag_list,enumerated_flag_list)

In [5]:
clf = SVC()

# train_features = train_set.drop(columns=['attack_type'])
train_target = train_set['attack_type']

# clf.fit(train_features, train_target)

In [6]:
test_set = pd.read_csv('../KDDTest+.csv', header=None)
test_set.columns = dfhd

test_set = test_set[cols]

test_set = test_set.replace(service_list2,enumerated_service_list)
test_set['service'] = test_set['service'].apply(lambda x: '-1' if type(x) is str else x)

test_set = test_set.replace(prot_list2, enumerated_prot_list)
test_set['protocol_type'] = test_set['protocol_type'].apply(lambda x: '-1' if type(x) is str else x)

test_set = test_set.replace(flag_list2, enumerated_flag_list)
test_set['flag'] = test_set['flag'].apply(lambda x: '-1' if type(x) is str else x)

# Mini test_set as not full training set used
# test_set = shuffle(test_set)
# test_set = test_set[:201]


# test_features = test_set.drop(columns=['attack_type'])
test_target = test_set['attack_type']

# pred_list = clf.predict(test_features)

# np.mean( pred_list == test_target)

In [7]:
# 2 Feature Wrapper

# TODO: Add Precision and Recall plus F-Score, other evaluation metrics

fields_list = fn[0].tolist()
fields_list.remove('attack_type')

fieldnum = len(fields_list)

result_df = pd.DataFrame(columns=['Feature 1','Feature 2', 'Accuracy 1', 'Accuracy 2', 'Accuracy 3'])


for i in range(fieldnum):
    for j in range(i+1, fieldnum):
        field1 = fields_list[i]
        field2 = fields_list[j]
     
        train_features = train_set[[field1, field2]]
        clf.fit(train_features, train_target)

        accuracy = []

        for x in range(3):
            test_set = shuffle(test_set)
            test_set_ins = test_set[:201]
            test_target = test_set_ins['attack_type']
            test_features = test_set_ins[[field1, field2]]
            pred_list = clf.predict(test_features)
            accuracy = accuracy + [np.mean( pred_list == test_target)]

        accuracies = [field1, field2] + accuracy
        newentry = pd.DataFrame([accuracies], columns=['Feature 1','Feature 2', 'Accuracy 1', 'Accuracy 2', 'Accuracy 3'])
        result_df = result_df.append(newentry, ignore_index=True)

result_df['Average Accuracy'] = result_df.mean(axis=1)
result_df.sort_values('Average Accuracy', ascending=False).to_csv('2 Features wrapper accuracy multi class classifier.csv')


In [8]:
# 3 Feature Wrapper

fields_list = fn[0].tolist()
fields_list.remove('attack_type')

result_df = pd.DataFrame(columns=['Feature 1','Feature 2', 'Feature 3', 'Accuracy 1', 'Accuracy 2', 'Accuracy 3'])

for i in range(fieldnum):
    for j in range(i+1, fieldnum):
        for k in range(j+1, fieldnum):
            field1 = fields_list[i]
            field2 = fields_list[j]
            field3 = fields_list[k]

            train_features = train_set[[field1, field2, field3]]
            clf.fit(train_features, train_target)

            accuracy = []

            for x in range(3):
                test_set = shuffle(test_set)
                test_set_ins = test_set[:201]
                test_target = test_set_ins['attack_type']
                test_features = test_set_ins[[field1, field2, field3]]
                pred_list = clf.predict(test_features)
                accuracy = accuracy + [np.mean( pred_list == test_target)]

            accuracies = [field1, field2, field3] + accuracy
            newentry = pd.DataFrame([accuracies], columns=['Feature 1','Feature 2', 'Feature 3', 'Accuracy 1', 'Accuracy 2', 'Accuracy 3'])
            result_df = result_df.append(newentry, ignore_index=True)

result_df['Average Accuracy'] = result_df.mean(axis=1)
result_df.sort_values('Average Accuracy', ascending=False).to_csv('3 Features wrapper accuracy multi class classifier.csv')


In [8]:
# 4 Feature Wrapper

fields_list = fn[0].tolist()
fields_list.remove('attack_type')

fieldnum = len(fields_list)

result_df = pd.DataFrame(columns=['Feature 1','Feature 2', 'Feature 3', 'Feature 4', 'Accuracy 1', 'Accuracy 2', 'Accuracy 3'])

for i in range(fieldnum):
    for j in range(i+1, fieldnum):
        for k in range(j+1, fieldnum):
            for l in range(k+1, fieldnum):
                field1 = fields_list[i]
                field2 = fields_list[j]
                field3 = fields_list[k]
                field4 = fields_list[l]
                
                train_features = train_set[[field1, field2, field3, field4]]
                clf.fit(train_features, train_target)

                accuracy = []

                for x in range(3):
                    test_set = shuffle(test_set)
                    test_set_ins = test_set[:201]
                    test_target = test_set_ins['attack_type']
                    test_features = test_set_ins[[field1, field2, field3, field4]]
                    pred_list = clf.predict(test_features)
                    accuracy = accuracy + [np.mean( pred_list == test_target)]

                accuracies = [field1, field2, field3, field4] + accuracy
                newentry = pd.DataFrame([accuracies], columns=['Feature 1','Feature 2', 'Feature 3', 'Feature 4', 'Accuracy 1', 'Accuracy 2', 'Accuracy 3'])
                result_df = result_df.append(newentry, ignore_index=True)

result_df['Average Accuracy'] = result_df.mean(axis=1)
result_df.sort_values('Average Accuracy', ascending=False).to_csv('4 Features wrapper accuracy multi class classifier.csv')


In [ ]:
# 5 Feature Wrapper

fields_list = fn[0].tolist()
fields_list.remove('attack_type')

result_df = pd.DataFrame(columns=['Feature 1','Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Accuracy 1', 'Accuracy 2', 'Accuracy 3'])

for i in range(fieldnum):
    for j in range(i+1, fieldnum):
        for k in range(j+1, fieldnum):
            for l in range(k+1, fieldnum):
                for m in range(l+1, fieldnum):
                    field1 = fields_list[i]
                    field2 = fields_list[j]
                    field3 = fields_list[k]
                    field4 = fields_list[l]
                    field5 = fields_list[m]

                    train_features = train_set[[field1, field2, field3, field4, field5]]
                    clf.fit(train_features, train_target)

                    accuracy = []

                    for x in range(3):
                        test_set = shuffle(test_set)
                        test_set_ins = test_set[:201]
                        test_target = test_set_ins['attack_type']
                        test_features = test_set_ins[[field1, field2, field3, field4, field5]]
                        pred_list = clf.predict(test_features)
                        accuracy = accuracy + [np.mean( pred_list == test_target)]

                    accuracies = [field1, field2, field3, field4, field5] + accuracy
                    newentry = pd.DataFrame([accuracies], columns=['Feature 1','Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Accuracy 1', 'Accuracy 2', 'Accuracy 3'])
                    result_df = result_df.append(newentry, ignore_index=True)

result_df['Average Accuracy'] = result_df.mean(axis=1)
result_df.sort_values('Average Accuracy', ascending=False).to_csv('5 Features wrapper accuracy multi class classifier.csv')
